In [131]:
import numpy as np
import pandas as pd
import tqdm
from sklearn import tree
from kmodes.kmodes import KModes
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import math 
import sklearn
import scipy.special as sp
from tqdm import tqdm
import SimpSOM as sps
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FastICA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
%matplotlib notebook

df=[]
df1=[]
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/b"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/f2"+str(i)+".csv"))

for i in range(7):
    df.append(pd.read_csv("/home/franc/franc/matches/ec"+str(i)+".csv"))
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/n"+str(i)+".csv"))
    
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/d2"+str(i)+".csv"))
    
#for i in range(6):
#    df.append(pd.read_csv("/home/franc/franc/matches/d1"+str(i)+".csv"))    
#df1.append(pd.read_csv("/home/franc/franc/matches/SWE.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/DNK.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/CHN.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/ARG.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/NOR.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/JPN.csv"))#Doesnt work well
#df1.append(pd.read_csv("/home/franc/franc/matches/AUT.csv"))
df1.append(pd.read_csv("/home/franc/franc/matches/POL.csv"))

df=pd.concat(df,sort=False)
#df1=pd.concat(df1,sort=False)
rat_dic={}
fet_set=[]
tar_set=[]

In [132]:
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 
  
  
# Function to calculate Elo rating 
# K is a constant. 
# d determines whether 
# Player A wins or Player B.  
def EloRating(Ra, Rb, K, d): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = nprob(Ra, Rb) 
    Pb = 1-Pa
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (d == 1) : 
        Ra = Ra + K * (1 - Pa) 
        Rb = Rb + K * (0 - Pb) 
      
  
    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    elif (d==0.5) : 
        Ra = Ra + K * (0.5 - Pa) 
        Rb = Rb + K * (0.5 - Pb) 
      
    else : 
        Ra = Ra + K * (0 - Pa) 
        Rb = Rb + K * (1 - Pb) 
      
    return Ra,Rb
    print("Updated Ratings:-") 
    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

def Probabilityad(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 

def nprob(rating1, rating2): 
  
    return 1/2*(1 + sp.erf((rating1-rating2)/(300*math.sqrt(2)))) 

def spreadelo(Ra, Rb, K, ga,gb,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga-gb > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def adelo(Ra, Rb, K, g,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if g > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def kgvelo(Ra, Rb, K, ga,gb): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga==0:
        Ra=Ra+K*(0-Pa)
    else:
        Ra=Ra+K*(1-Pa)
        
    if gb==0:
        Rb=Rb+K*(0-Pb)
    else:
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

In [134]:
df['Date']=pd.to_datetime(df['Date']
                              ,infer_datetime_format=True)
last_two=df[(df["Date"].dt.year>2012) & (df["Date"].dt.year<2018)]
for i in df.HomeTeam.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test=df[(df["Date"].dt.year>=2014) & (df["Date"].dt.year<2019)]

for i in range(len(last_two)):
    rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"],60,last_two.iloc[i,4],last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"],60,last_two.iloc[i,4],last_two.iloc[i,5],1)
    rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"],60,last_two.iloc[i,4],last_two.iloc[i,5],2)
    rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"],60,last_two.iloc[i,5],last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"],60,last_two.iloc[i,5],last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"]=adelo(rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"],60,last_two.iloc[i,4],0)
    rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"]=adelo(rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"],60,last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"]=adelo(rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"],60,last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"]=adelo(rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"],60,last_two.iloc[i,5],1)    
    rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"]=adelo(rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"],60,last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"]=adelo(rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"],60,last_two.iloc[i,5],2)     
    rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"]=kgvelo(rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"],60,last_two.iloc[i,4],last_two.iloc[i,5])
    rat_dic[last_two.iloc[i,2]]["nm"]+=1
    rat_dic[last_two.iloc[i,3]]["nm"]+=1
    

#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr"],rat_dic[test.iloc[t,3]]["tr"]))   
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr1"],rat_dic[test.iloc[t,3]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr2"],rat_dic[test.iloc[t,3]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr1"],rat_dic[test.iloc[t,2]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr2"],rat_dic[test.iloc[t,2]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar"],rat_dic[test.iloc[t,3]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar"],rat_dic[test.iloc[t,2]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar1"],rat_dic[test.iloc[t,3]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar1"],rat_dic[test.iloc[t,2]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar2"],rat_dic[test.iloc[t,3]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar2"],rat_dic[test.iloc[t,2]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["kgv"],rat_dic[test.iloc[t,3]]["kgv"]))
    tmp1.append(test.iloc[t,4])
    tmp1.append(test.iloc[t,5])
    tmp1.append(test.iloc[t,4]+test.iloc[t,5])
    tmp1.append(test.iloc[t,4]-test.iloc[t,5])
    tmp1.append((test.iloc[t,4]!=0) and (test.iloc[t,5]!=0))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>1.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>2.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))

In [83]:
df1['Date']=pd.to_datetime(df1['Date']
                              ,infer_datetime_format=True)
last_two1=df1[(df1["Date"].dt.year>2012) & (df1["Date"].dt.year<2018)]
for i in df1.Home.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

for i in df1.Away.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test1=df1[(df1["Date"].dt.year>=2014) & (df1["Date"].dt.year<2019)]

for i in range(len(last_two1)):
    rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],1)
    rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],2)
    rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"],60,last_two1.iloc[i,7],0)
    rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"],60,last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"],60,last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"],60,last_two1.iloc[i,8],1)    
    rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"],60,last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"],60,last_two1.iloc[i,8],2)     
    rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"]=kgvelo(rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"],60,last_two1.iloc[i,7],last_two1.iloc[i,8])
    rat_dic[last_two1.iloc[i,5]]["nm"]+=1
    rat_dic[last_two1.iloc[i,6]]["nm"]+=1
    
#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test1)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr"],rat_dic[test1.iloc[t,6]]["tr"]))   
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr1"],rat_dic[test1.iloc[t,6]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr2"],rat_dic[test1.iloc[t,6]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr1"],rat_dic[test1.iloc[t,5]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr2"],rat_dic[test1.iloc[t,5]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar"],rat_dic[test1.iloc[t,6]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar"],rat_dic[test1.iloc[t,5]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar1"],rat_dic[test1.iloc[t,6]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar1"],rat_dic[test1.iloc[t,5]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar2"],rat_dic[test1.iloc[t,6]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar2"],rat_dic[test1.iloc[t,5]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["kgv"],rat_dic[test1.iloc[t,6]]["kgv"]))
    tmp1.append(test1.iloc[t,7])
    tmp1.append(test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]+test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]-test1.iloc[t,8])
    tmp1.append((test1.iloc[t,7]!=0) and (test1.iloc[t,8]!=0))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>1.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>2.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))



In [141]:
n=40
for i in range(n):
    t1=df[(df["Date"].dt.year==2019) ].iloc[i,2]
    t2=df[(df["Date"].dt.year==2019) ].iloc[i,3]
    tmp=[]
    
    tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
    tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
    tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
    tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
    tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
    tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
    tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
    tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
    tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
    tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
    tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
    tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
    tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
    fet_set.append(tmp)
    tmp1.append(test.iloc[i,4])
    tmp1.append(test.iloc[i,5])
    tmp1.append(test.iloc[i,4]+test.iloc[i,5])
    tmp1.append(test.iloc[i,4]-test.iloc[i,5])
    tmp1.append((test.iloc[i,4]!=0) and (test.iloc[i,5]!=0))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>1.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>2.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>3.5))
    tar_set.append(np.array(tmp1))
    print(t1,t2,tmp1,i)

Aston Villa QPR [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 0
Blackburn West Brom [2.0, 2.0, 4.0, 0.0, True, 1, 1, 1] 1
Brentford Norwich [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 2
Derby Middlesbrough [3.0, 2.0, 5.0, 1.0, True, 1, 1, 1] 3
Hull Bolton [0.0, 1.0, 1.0, -1.0, False, 0, 0, 0] 4
Ipswich Millwall [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 5
Nott'm Forest Leeds [1.0, 2.0, 3.0, -1.0, True, 1, 1, 0] 6
Reading Swansea [1.0, 3.0, 4.0, -2.0, True, 1, 1, 1] 7
Rotherham Preston [2.0, 1.0, 3.0, 1.0, True, 1, 1, 0] 8
Sheffield Weds Birmingham [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 9
Stoke Bristol City [2.0, 0.0, 2.0, 2.0, False, 1, 0, 0] 10
Wigan Sheffield United [4.0, 1.0, 5.0, 3.0, True, 1, 1, 1] 11
Leeds Derby [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 12
Birmingham Middlesbrough [0.0, 2.0, 2.0, -2.0, False, 1, 0, 0] 13
Brentford Stoke [1.0, 1.0, 2.0, 0.0, True, 1, 0, 0] 14
Bristol City Bolton [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 15
Hull Sheffield Weds [1.0, 0.0, 1.0, 1.0, False, 0, 0, 0] 16
Ipswich Rotherham [

In [69]:
n=40
for i in range(n):
    t1=df1[(df1["Date"].dt.year==2019) ].iloc[i,5]
    t2=df1[(df1["Date"].dt.year==2019) ].iloc[i,6]
    tmp=[]
    
    tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
    tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
    tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
    tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
    tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
    tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
    tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
    tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
    tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
    tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
    tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
    tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
    tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
    fet_set.append(tmp)
    tmp1.append(test1.iloc[i,7])
    tmp1.append(test1.iloc[i,8])
    tmp1.append(test1.iloc[i,7]+test1.iloc[i,8])
    tmp1.append(test1.iloc[i,7]-test1.iloc[i,8])
    tmp1.append((test1.iloc[i,7]!=0) and (test1.iloc[i,8]!=0))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>1.5))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>2.5))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>3.5))
    tar_set.append(np.array(tmp1))
    print(t1,t2,tmp1,i)

Legnica Jagiellonia [1, 0, 1, 1, False, 0, 0, 0] 0
Lech Poznan Zaglebie [1, 0, 1, 1, False, 0, 0, 0] 1
Slask Wroclaw Zaglebie Sosnowiec [0, 2, 2, -2, False, 1, 0, 0] 2
Cracovia Piast Gliwice [2, 3, 5, -1, True, 1, 1, 1] 3
Lechia Gdansk Pogon Szczecin [1, 0, 1, 1, False, 0, 0, 0] 4
Arka Gdynia Korona Kielce [3, 0, 3, 3, False, 1, 1, 0] 5
Plock Legia [2, 1, 3, 1, True, 1, 1, 0] 6
Gornik Z. Wisla [0, 1, 1, -1, False, 0, 0, 0] 7
Pogon Szczecin Gornik Z. [2, 2, 4, 0, True, 1, 1, 1] 8
Piast Gliwice Lech Poznan [5, 1, 6, 4, True, 1, 1, 1] 9
Jagiellonia Plock [1, 1, 2, 0, True, 1, 0, 0] 10
Zaglebie Sosnowiec Arka Gdynia [0, 3, 3, -3, False, 1, 1, 0] 11
Korona Kielce Lechia Gdansk [1, 1, 2, 0, True, 1, 0, 0] 12
Zaglebie Legnica [3, 1, 4, 2, True, 1, 1, 1] 13
Legia Cracovia [2, 3, 5, -1, True, 1, 1, 1] 14
Wisla Slask Wroclaw [0, 3, 3, -3, False, 1, 1, 0] 15
Arka Gdynia Piast Gliwice [2, 1, 3, 1, True, 1, 1, 0] 16
Korona Kielce Pogon Szczecin [0, 0, 0, 0, False, 0, 0, 0] 17
Gornik Z. Zaglebie Sos

In [139]:
#asa=list(df1.Away.unique())
asa=list(df.AwayTeam.unique())

td={}
for i in range(len(asa)):
    td[asa[i]]=i
for i in td:
    print(i,td[i])

Huddersfield 0
Middlesbrough 1
Charlton 2
Bolton 3
Watford 4
Sheffield Weds 5
Brighton 6
Blackburn 7
Wolves 8
Peterboro 9
Blackpool 10
Bristol City 11
Leeds 12
Derby 13
Cardiff 14
Crystal Palace 15
Leicester 16
Nott'm Forest 17
Burnley 18
Millwall 19
Birmingham 20
Ipswich 21
Barnsley 22
Hull 23
Wigan 24
Yeovil 25
Reading 26
QPR 27
Bournemouth 28
Doncaster 29
Rotherham 30
Fulham 31
Norwich 32
Brentford 33
nan 34
Milton Keynes Dons 35
Preston 36
Newcastle 37
Burton 38
Aston Villa 39
Sheffield United 40
Sunderland 41
Swansea 42
Stoke 43
West Brom 44


In [140]:
fet_set=list(fet_set)[:-n]
tar_set=list(tar_set)[:-n]

In [129]:
n=1
t1=asa[27]
t2=asa[26]
tmp=[]
    
tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
fet_set.append(tmp)
tmp1=np.array([0,0,0,0,0,0,0,1])
tar_set.append(np.array(tmp1))
print(t1,t2,tmp1,i)

Valenciennes Nancy [0 0 0 0 0 0 0 1] 13


In [112]:
fet_set=np.array(fet_set)
tar_set=np.array(tar_set)
nc=200
f=[[[[] for i in range(nc)] for j in range(3)]for i in range(4)]
ft=[]

In [27]:
n=41

In [62]:
len(fet_set),len(tar_set),len(test1),n

(946, 946, 906, 40)

In [142]:
fet_set=np.array(fet_set)
tar_set=np.array(tar_set)
nc=200
f=[[[[] for i in range(nc)] for j in range(3)]for i in range(4)]
ft=[]
rft=[]
d1=[]
d2=[]
d=[]
d3=[]
for j in tqdm(range(1,1+n)):
    ft=[]
    f=[[[[] for i in range(nc)] for j in range(3)]for i in range(4)]
    pca = PCA(n_components=4)
    l=pca.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[0][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[0][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[0][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = KernelPCA(n_components=3, kernel="cosine")
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[1][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[1][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[1][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = KernelPCA(n_components=3, kernel="rbf")
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[2][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[2][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[2][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]

    transformer = FastICA(n_components=4,random_state=0,tol=1)
    l=transformer.fit_transform(fet_set[:,:12])
    kmeans = KMeans(n_clusters=nc, random_state=0).fit(l)
    clustering = AgglomerativeClustering(n_clusters=nc,linkage="ward").fit(l)
    clustering1 = AgglomerativeClustering(n_clusters=nc,linkage="complete").fit(l)

    for i in range(len(kmeans.labels_)-1):
        f[3][0][kmeans.labels_[i]].append(tar_set[i])
    for i in range(len(clustering.labels_)-1):
        f[3][1][clustering.labels_[i]].append(tar_set[i])
    for i in range(len(clustering1.labels_)-1):
        f[3][2][clustering1.labels_[i]].append(tar_set[i])
    
    ft+=[kmeans.labels_[-1*j],clustering.labels_[-1*j],clustering1.labels_[-1*j]]
    d3+=[tar_set[-j]]
    tm2=[]
    tm1=[]
    tm=[]
    for k in range(len(ft)):
        #print(int(k/3),(k-3*int(k/3))%3,ft[k],np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]]))
        tm+=[np.array([np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]]) ])]
        tm1+=[np.array([np.array(f[int(k/3)][(k-3*int(k/3))%3][ft[k]])[:,u].mean() for u in range(len(tar_set[0]))])]
        tm2+=[len(f[int(k/3)][(k-3*int(k/3))%3][ft[k]])]
        #print("-------------------------------------------------")
    d+=[tm]
    d1+=[tm1]
    d2+=[tm2]

for ki in range(n):
    ff=[]
    #ki=20
    g=-10
    transformer = KernelPCA(n_components=4, kernel="rbf")
    l=transformer.fit_transform(fet_set[:,:8])
    pw=sklearn.metrics.pairwise.cosine_similarity(l)
    ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
    pw=sklearn.metrics.pairwise.laplacian_kernel(l)
    ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
    for i in range(12):
        ff.append([[d[-1-ki][i][0][u][p] for p in range(len(d[-1-ki][i][0][u]))] for u in range(len(d[-1-ki][i][0]))])
    dic={}
    dic1={}
    for i in range(len(ff)):
        for j in range(len(ff[i])):
            if(str(ff[i][j]) not in dic):
                dic[str(ff[i][j])]=ff[i][j]
            if(str(ff[i][j]) not in dic1):
                dic1[str(ff[i][j])]=1
            else:
                dic1[str(ff[i][j])]+=1
#print(len(ff),dic1)            
    print([[dic[i],dic1[i]] for i in [k for (k,v) in dic1.items() if v >= (8/5)*len(ff)]],"|||",len(ff),"\n --------- \n",d3[-1-ki],"\n ======= \n")


100%|██████████| 40/40 [04:09<00:00,  6.07s/it]


[[[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 51], [[1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0], 46], [[2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0], 35], [[2.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0], 23]] ||| 14 
 --------- 
 [1. 1. 2. 0. 1. 1. 0. 0.] 

[[[1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0], 27], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 23], [[2.0, 2.0, 4.0, 0.0, 1.0, 1.0, 1.0, 1.0], 37]] ||| 14 
 --------- 
 [2. 2. 4. 0. 1. 1. 1. 1.] 

[[[2.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 0.0], 34], [[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 67], [[2.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0], 64], [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 49], [[3.0, 2.0, 5.0, 1.0, 1.0, 1.0, 1.0, 1.0], 27], [[1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0], 45], [[3.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 1.0], 30], [[0.0, 1.0, 1.0, -1.0, 0.0, 0.0, 0.0, 0.0], 44]] ||| 14 
 --------- 
 [1. 1. 2. 0. 1. 1. 0. 0.] 

[[[0.0, 1.0, 1.0, -1.0, 0.0, 0.0, 0.0, 0.0], 27], [[2.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0], 63], [[3.0, 1.0, 4.0, 2.0, 1.0, 1.0,

[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 23], [[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 32], [[2.0, 0.0, 2.0, 2.0, 0.0, 1.0, 0.0, 0.0], 36], [[1.0, 2.0, 3.0, -1.0, 1.0, 1.0, 1.0, 0.0], 34]] ||| 14 
 --------- 
 [2. 0. 2. 2. 0. 1. 0. 0.] 

[[[2.0, 2.0, 4.0, 0.0, 1.0, 1.0, 1.0, 1.0], 29]] ||| 14 
 --------- 
 [2. 2. 4. 0. 1. 1. 1. 1.] 

[[[1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 0.0, 0.0], 25], [[3.0, 0.0, 3.0, 3.0, 0.0, 1.0, 1.0, 0.0], 26], [[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 25]] ||| 14 
 --------- 
 [ 1.  2.  3. -1.  1.  1.  1.  0.] 



In [40]:
for i in range(len(d2)):
    for j in range(len(d2[i])):
        d1[i][j]*=(d2[i][j]/sum(d2[i]))
for i in range(len(d1)):
    d1[i]=np.transpose(d1[i])

In [111]:
for i in range(len(f)):
    for j in range(len(f[i])):
        for k in range(len(f[i][j])):
            if f[i][j][k]==[]:
                print(i,j,k)

In [114]:
for ki in range(n):
    ff=[]
    #ki=20
    g=-10
    transformer = KernelPCA(n_components=4, kernel="rbf")
    l=transformer.fit_transform(fet_set[:,:8])
    pw=sklearn.metrics.pairwise.cosine_similarity(l)
    ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
    pw=sklearn.metrics.pairwise.laplacian_kernel(l)
    ff.append(list(map(list,\
                   list(tar_set[pw[-1-ki].argsort()[g:][::-1][1:]]))))
    for i in range(12):
        ff.append([[d[-1-ki][i][0][u][p] for p in range(len(d[-1-ki][i][0][u]))] for u in range(len(d[-1-ki][i][0]))])
    dic={}
    dic1={}
    for i in range(len(ff)):
        for j in range(len(ff[i])):
            if(str(ff[i][j]) not in dic):
                dic[str(ff[i][j])]=ff[i][j]
            if(str(ff[i][j]) not in dic1):
                dic1[str(ff[i][j])]=1
            else:
                dic1[str(ff[i][j])]+=1
#print(len(ff),dic1)            
    print([[dic[i],dic1[i]] for i in [k for (k,v) in dic1.items() if v >= (3/2)*len(ff)]],"|||",len(ff),"\n --------- \n",d3[-1-ki],"\n ======= \n")

[[[1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], 41]] ||| 14 
 --------- 
 [0. 0. 0. 0. 0. 0. 0. 1.] 



In [280]:
len(fet_set),(df["Date"].dt.year>=2014).sum()

(1220, 1272)

In [99]:
ft

[53, 179, 143, 39, 183, 193, 62, 183, 175, 54, 167, 191]